In [1]:
from datetime import date, timedelta, datetime
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import os
from functions import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import csv
import numpy as np
import pandas as pd
import unidecode
from tqdm import tqdm
from fake_useragent import UserAgent  


# Khởi tạo fake UserAgent
ua = UserAgent()

# Cấu hình với fake User-Agent
options = Options()
user_agent = ua.random  # Lấy một User-Agent ngẫu nhiên
options.add_argument(f"user-agent={user_agent}")

# Mở trình duyệt với undetected-chromedriver
driver = uc.Chrome(driver_executable_path=ChromeDriverManager().install(), options=options)
driver.get('https://www.tripadvisor.com')
time.sleep(random.uniform(3, 5))  # Thời gian trễ ngẫu nhiên
driver.maximize_window()

In [4]:
# Hàm chuyển đổi URL để thêm '-oa0-' vào giữa
def convert_tripadvisor_url(url):
    parts = url.split('-Activities-')
    if len(parts) != 2:
        return None  # Trả về None nếu URL không hợp lệ
    return f"{parts[0]}-Activities-oa0-{parts[1]}"

# Tìm và nhấn vào tìm kiếm điểm du lịch với XPath
attraction_ele = driver.find_element(By.XPATH, "//a[@href='/Attractions']")
attraction_ele.click()

# Nhập thành phố
country = 'Scotland'
#country = 'England'
#country = 'Wales'
#country = 'North Ireland'
place = 'Aberdeen'
time.sleep(random.uniform(0.8, 2))  # Thời gian trễ ngẫu nhiên

# Tìm thanh search
search_bar_location = driver.find_element(By.XPATH, 
    "/html/body/div[1]/main/div[4]/div/div/div[2]/div/form/div[1]/div/input")

# Thêm thời gian trễ ngẫu nhiên giữa các hành động
time.sleep(random.uniform(0.8, 2))
search_bar_location.clear()
time.sleep(random.uniform(0.8, 2))
search_bar_location.send_keys(place)

# Di chuyển chuột đến thanh tìm kiếm để giả lập người dùng thật
actions = ActionChains(driver)
actions.move_to_element(search_bar_location).perform()
time.sleep(random.uniform(0.8, 2))

# Tìm liên kết đầu tiên trong kết quả và nhấn vào
attraction_link = driver.find_element(By.XPATH, 
    "/html/body/div[@id='lithium-root']/main/div[4]/div/div[@class='upkHE F1']/div[@class='ctKgY']/div/form/div/div[@id='typeahead_results']/a[1]")
time.sleep(random.uniform(2, 4))
attraction_link.click()
time.sleep(random.uniform(3, 5))  # Chờ trang load

# Lấy URL hiện tại
current_url = driver.current_url

# Chuyển đổi URL với hàm mới
new_url = convert_tripadvisor_url(current_url)
if new_url:
    print(f"Chuyển sang URL mới: {new_url}")
    driver.get(new_url)  # Mở URL đã chuyển đổi

time.sleep(random.uniform(2, 4))  # Chờ để xem nội dung trang mới


Chuyển sang URL mới: https://www.tripadvisor.com/Attractions-g186487-Activities-oa0-Aberdeen_Aberdeenshire_Scotland.html


In [3]:
def getURL():
    # Parse HTML từ trình duyệt
    page_source = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Kiểm tra lại class và tìm các thẻ <a> có thuộc tính href
    attractions = page_source.find_all('a', href=True, target='_blank')
    all_attraction_url = []

    for attraction in attractions:
        attraction_link = attraction.get('href')
        # Kiểm tra nếu URL hợp lệ và không chứa '#'
        if attraction_link and '#' not in attraction_link:
            full_url = 'https://www.tripadvisor.com' + attraction_link
            if full_url not in all_attraction_url:
                all_attraction_url.append(full_url)

    return all_attraction_url

# Gọi hàm và in kết quả
getURL()



['https://www.tripadvisor.com/Attraction_Review-g186487-d210323-Reviews-Aberdeen_Maritime_Museum-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.comhttps://tripadvisor.co1.qualtrics.com/jfe/form/SV_dnbpJe7kYaoZMlE?h=210323&g=186487',
 'https://www.tripadvisor.com/ClientLink?value=Z1B5Xy9BdHRyYWN0aW9uX1Byb2R1Y3RzLWcxODY0ODctZDIxMDMyMy1BYmVyZGVlbl9NYXJpdGltZV9NdXNldW0tQWJlcmRlZW5fQWJlcmRlZW5zaGlyZV9TY290bGFuZC5odG1sX3lVQw%3D%3D',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d211634-Reviews-Footdee_Fittie-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.comhttps://tripadvisor.co1.qualtrics.com/jfe/form/SV_dnbpJe7kYaoZMlE?h=211634&g=186487',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d206562-Reviews-Slains_Castle-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.comhttps://tripadvisor.co1.qualtrics.com/jfe/form/SV_dnbpJe7kYaoZMlE?h=206562&g=186487',
 'https://www.tripadvisor.com/ClientLink?value=WklBXy9BdHRyYWN0aW9uX1B

In [4]:
# Hàm lấy URL của tất cả các trang
URLs_all_page = []

def getUrlOnPages():
    input_page = 1000

    for page in range(input_page):
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  
            last_height = new_height

        URLs_one_page = getURL()  
        # URLs_all_page.extend(URLs_one_page)  # Thêm URL của trang hiện tại vào danh sách tổng
        
        
        # Kiểm tra và chỉ thêm những URL chưa có trong danh sách
        for url in URLs_one_page:
            if 'Attraction' in url and url not in URLs_all_page:
                URLs_all_page.append(url)
                
        try:
            next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
            next_page_url = next_button.get_attribute('href')  
            driver.get(next_page_url)  
            time.sleep(random.uniform(2, 4))
        except:
            print(f"Không tìm thấy nút 'next' ở trang {page + 1}. Dừng scrape.")
            break  

    return URLs_all_page

# Gọi hàm
getUrlOnPages()


Không tìm thấy nút 'next' ở trang 13. Dừng scrape.


['https://www.tripadvisor.com/Attraction_Review-g186487-d210323-Reviews-Aberdeen_Maritime_Museum-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d211634-Reviews-Footdee_Fittie-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d206562-Reviews-Slains_Castle-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d211625-Reviews-Duthie_Park_Winter_Gardens-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d211661-Reviews-The_Gordon_Highlanders_Museum-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d1728772-Reviews-St_Machar_s_Cathedral-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d6460164-Reviews-Northlink_Ferries_Aberdeen-Aberdeen_Aberdeenshire_Scotland.html',
 'https://www.tripadvisor.com/Attraction_Review-g186487-d1901

In [5]:
len(URLs_all_page)

389

In [6]:
import pandas as pd
df = pd.read_csv("Restaurant_Data\Restaurant_England.csv", 
                 on_bad_lines='skip',   
                 delimiter=',', 
                 encoding='utf-8-sig')
df["Tourist Attraction"].unique()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17544\1537383061.py:2: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Restaurant_Data\Restaurant_England.csv",


array(['Southampton', 'Wolverhampton', 'Plymouth', 'Stoke-On-Trent',
       'Belfast', 'Nottingham', 'Bradford', 'Aberdeen', 'Armagh',
       'Bangor', 'Bath And North East Somerset', 'Brighton And Hove',
       'London', 'Cardiff', 'Coventry', 'Derby', 'Derry', 'Doncaster',
       'Dundee', 'Durham', 'Lisburn', 'Glasgow', 'Edinburgh',
       'Kingston Upon Hull', 'Essex', 'Leeds', 'Newport', 'Newry',
       'Perth', 'Liverpool', 'Newcastle upon Tyne', 'Kent', 'Hampshire',
       'Bỉrmingham', 'Manchester', 'Sheffield', 'Peterborough', 'Salford',
       'Southend-on-Sea', 'Stirling', 'Sunderland', 'Wakefield',
       'Lancashire', 'York', 'Swansea', 'Dudley', 'Sandwell', 'Devon',
       'Buckinghamshire', 'Cambridgeshire', 'Cumbria', 'Derbyshire',
       'East Sussex', 'Dorset', 'North Yorkshire', 'Nottinghamshire',
       'Gloucestershire', 'Hertfordshire', 'Surrey', 'Leicestershire',
       'Lincolnshire', 'Norfolk', 'Oxfordshire', 'Somerset',
       'Staffordshire'], dtype=object)

In [13]:
# Kiểm tra xem file đã có dữ liệu chưa (để tránh ghi lại tiêu đề)
file_exists = os.path.isfile("Attraction_Data/Attraction_Data_UK.csv")

with open(f"Attraction_Data/Attraction_Data_UK.csv", 'a', newline='', encoding='utf-8-sig') as file_output:
    headers = []
    
    writer = csv.DictWriter(file_output, delimiter=",", lineterminator="\n", fieldnames=headers)

    # Ghi tiêu đề chỉ khi file trống (chưa có dữ liệu)
    if not file_exists:
        writer.writeheader()

    # Thiết lập thanh tiến độ
    with tqdm(total=len(URLs_all_page), desc="Crawling restaurant data", dynamic_ncols=True, unit="restaurant", leave=False) as pbar:
        for attraction_url in URLs_all_page:
            driver.get(attraction_url)
            time.sleep(random.uniform(0.1, 0.15))
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
        
            # Lấy thông tin tên 
            title_div = soup.find('div', {'class': 'mmBWG'})
            if title_div:
                title_loc = title_div.find_all('div')
                if title_loc:
                    h1_tag = title_loc[0].find('h1')
                    if h1_tag and h1_tag.get_text():
                        name = h1_tag.get_text().strip()  
            time.sleep(random.uniform(0.2, 0.28))
            


            # Lấy số lượng đánh giá
            review_count_tag = soup.find('span', {'data-automation': 'reviewCount'})
            review_count = review_count_tag.get_text(strip=True).split()[0] if review_count_tag else np.nan

            # Lấy số sao 
            rating_tag = soup.find('svg', {'data-automation': 'reviewBubbles'}).find('title')
            rating_value = rating_tag.get_text(strip=True).split()[0] if rating_tag else np.nan


            rankings = soup.select('span.Wb > a')

            for a in rankings:
                ranking_info = a.get_text(strip=True) if a else np.nan
                print(f"Xếp hạng: {ranking_info}")

            
            
            # Di chuyển chuột đến giữa trang
            ActionChains(driver).move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()

            # Cuộn xuống cuối trang
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            
            
  
                        
            # Ghi dữ liệu vào CSV
            writer.writerow({

            })
            
            
            # Cập nhật thanh tiến độ sau mỗi nhà hàng
            pbar.update(1)


KeyboardInterrupt: 